In [73]:
import numpy as np
import pandas as pd
import os
import torch
import matplotlib.pyplot as plt
from torchvision.models import resnet50 
import preprocess.load_split_data as pr
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Subset

## load data and model 

In [74]:
# Load MNIST dataset
transform_mnist=transforms.Compose([
    transforms.ToTensor(),
    transforms.ColorJitter(),
    # transforms.Normalize((0.1307,), (0.3081,))
])
class_mnist=pr.LoadDataset("MNIST",transform=transform_mnist)
mnist=class_mnist.load_dataset()

In [75]:
mnist

(Dataset MNIST
     Number of datapoints: 60000
     Root location: ./datasets
     Split: Train
     StandardTransform
 Transform: Compose(
                ToTensor()
                ColorJitter(brightness=None, contrast=None, saturation=None, hue=None)
            ),
 Dataset MNIST
     Number of datapoints: 10000
     Root location: ./datasets
     Split: Test
     StandardTransform
 Transform: Compose(
                ToTensor()
                ColorJitter(brightness=None, contrast=None, saturation=None, hue=None)
            ))

In [38]:
train_mnist,test_mnist=mnist

In [39]:
train_mnist.data

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [40]:
# Setup the batch size hyperparameter
BATCH_SIZE = 64

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_mnist, # dataset to turn into iterable
    batch_size=BATCH_SIZE,# how many samples per batch?
    shuffle=True,
)

test_dataloader = DataLoader(test_mnist,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [41]:
# Load the pre-trained ResNet50 model without pre-trained weights
model = resnet50(pretrained=False)

/home/nasserali/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nasserali/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [42]:
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Conversion en 3 canaux
model.fc = torch.nn.Linear(2048, 10)  # Modifier la couche de classification pour 10 classes

In [43]:
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [44]:
# Define the loss function
loss_function = torch.nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())


In [45]:
# Train the model
for epoch in range(10):
    for batch in train_dataloader:
        # Get the inputs and labels
        inputs, labels = batch

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = loss_function(outputs, labels)

        # Backpropagate
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluate the model
    with torch.no_grad():
        correct = 0
        total = 0
        for batch in test_dataloader:
            inputs, labels = batch
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total

        print(f"Epoch {epoch + 1}: Accuracy {accuracy}")


Epoch 1: Accuracy 0.9731
Epoch 2: Accuracy 0.9847
Epoch 3: Accuracy 0.9801
Epoch 4: Accuracy 0.9883
Epoch 5: Accuracy 0.9881
Epoch 6: Accuracy 0.9847
Epoch 7: Accuracy 0.9867
Epoch 8: Accuracy 0.9854
Epoch 9: Accuracy 0.9887
Epoch 10: Accuracy 0.9869


In [46]:
# save_folder='./models/model-resnet50_mnist'
# model_path = os.path.join(save_folder)
# torch.save(model.state_dict(), model_path)

In [47]:
model_state_dict = torch.load("./models/model-resnet50_mnist")

In [48]:
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [49]:
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [50]:

train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch



(torch.Size([64, 1, 28, 28]),
 tensor([8, 3, 0, 3, 2, 5, 9, 1, 1, 6, 5, 3, 9, 7, 1, 6, 2, 9, 6, 3, 3, 5, 4, 5,
         3, 0, 5, 4, 8, 2, 7, 4, 8, 4, 2, 9, 6, 3, 1, 2, 1, 2, 1, 3, 5, 8, 9, 0,
         2, 6, 4, 4, 7, 7, 3, 8, 6, 5, 1, 6, 4, 2, 2, 5]))

In [59]:
train_dataloader.dataset[0][0].shape

torch.Size([1, 28, 28])

In [67]:
torch.unsqueeze(train_dataloader.dataset[0][0],0).shape

torch.Size([1, 1, 28, 28])

In [68]:
pred_1=model(torch.unsqueeze(train_dataloader.dataset[0][0],0))

In [70]:
torch.argmax(pred_1)

tensor(5)

In [51]:
pred=model(train_features_batch)

In [52]:
pred[2]

tensor([12.6767, -6.9157, -0.5144, -5.4781, -6.1168, -2.9224, -3.2527, -2.2623,
        -0.7129,  0.3930], grad_fn=<SelectBackward0>)

In [54]:
torch.argmax(pred[1]
             )

tensor(3)

In [72]:
np.random.randint(5,8,(5,5,3))

array([[[6, 5, 5],
        [5, 5, 6],
        [6, 6, 5],
        [6, 7, 5],
        [6, 5, 6]],

       [[7, 6, 6],
        [7, 7, 6],
        [5, 5, 5],
        [5, 6, 7],
        [7, 6, 6]],

       [[6, 5, 6],
        [6, 7, 5],
        [6, 7, 6],
        [7, 7, 7],
        [6, 7, 6]],

       [[7, 6, 6],
        [7, 6, 7],
        [6, 7, 5],
        [6, 6, 5],
        [6, 6, 7]],

       [[6, 5, 7],
        [7, 6, 6],
        [5, 7, 7],
        [6, 7, 5],
        [7, 7, 5]]])

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),
#     tf.keras.layers.Conv1D(filters=16, kernel_size=3, padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling1D(pool_size=2),
#     tf.keras.layers.Dropout(rate = 0.5),
#     tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling1D(pool_size=2),
#     tf.keras.layers.Dropout(rate = 0.5),
#     tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'),
#     tf.keras.layers.MaxPooling1D(pool_size=2),
#     tf.keras.layers.Dropout(rate = 0.5),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=128, activation='relu'),
#     tf.keras.layers.Dense(units=num_of_labels, activation='sigmoid')
# ])